In [1]:
# -*- coding: utf-8 -*-
"""
Created on Tue Apr 21 12:34:54 2020

@author: guill
"""
import mysql.connector
import numpy as np
import pandas as pd
import os

mydb = mysql.connector.connect(
  port = 3309,
  host="127.0.0.1",
  user="root",
  passwd="",
  database = 'nautiatoolkit'
)
cursor = mydb.cursor()

finalpath = "C:/Users/guill/Documents/Universidad/PlataformaRefugiados/NAUTIA/DesarrolloPy/DataSetFinales"

def getPath(mainpath,filename):
    return os.path.join(mainpath, filename)

    
def concatDF(df1,df2):
    return  pd.concat([df1,df2],axis = 1, ignore_index = True, sort = True)

def mkCSV(df,fileName):
    df = df.dropna(how = 'all')
    df *= 1   
    fileName = fileName.lower()
    df.to_csv('DataSetFinales/'+fileName,sep=',',header = False, index=False, encoding='utf-8')

def is_non_zero_file(fpath):
    return os.path.isfile(fpath) and os.path.getsize(fpath)

def getTableName(elem):
    if(elem.find("_has_camp") != -1):
        x = elem.replace("_has_camp","")
    else:
        if(elem.find("_has_country") != -1):
            x = elem.replace("_has_country","")
        else:
            if(elem.find("_has_community") != -1):
                x = elem.replace("_has_community","")
    return x
def serviceTable(cad):
    result = False
    if(cad == "s_educationalcenter"):
        result = True
    else:
        if(cad == "s_primaryattention"):
            result = True
        else:
            if(cad == "s_hospital"):
                result = True
            else:
                if(cad == "s_othercenter"):
                    result = True
    return result

def isEducationalCenter(cad):
    result = False
    if(cad == "s_educationalcenter"):
        result = True           
    return result

def isOtherCenter(cad):
    result = False
    if(cad == "s_othercenter"):
        result = True           
    return result

def get_communityPK(elem):
    if(elem.find("_has_camp") != -1):
        cursor.execute("SELECT idCamp FROM camp ORDER BY idCamp DESC LIMIT 1")
    else:
        if(elem.find("_has_country") != -1):
            cursor.execute("SELECT idCountry FROM Country ORDER BY idCountry DESC LIMIT 1")
        else:
            if(elem.find("_has_community") != -1):
                cursor.execute("SELECT idCommunity FROM community ORDER BY idCommunity DESC LIMIT 1")
    return cursor.fetchall()

def get_tableFK(table):
    cursor.execute("SELECT id"+table+" FROM "+table+" ORDER BY id"+table+" DESC LIMIT 1")
    return cursor.fetchall()

def replacestr(df,cad1,cad2):
    cols=list(df.columns)
    for col in cols:
        df[col] = df[col].astype(str).str.replace(cad1,cad2)
    return df

def uniFormatTable(df):
    df = replacestr(df,"\r","")
    df = replacestr(df,"NaN","nan")
    df = replacestr(df,"None","nan")
    return df

def uniFormatDF(df):
    df = replacestr(df,"NaN","nan")
    df = replacestr(df,"None","nan")
    return df

def get_specialTableFKs(table,tableHas,x,y):
    cursor.execute("SELECT * FROM "+table)
    df1 = uniFormatTable(pd.DataFrame(cursor.fetchall()))
    df2 = pd.read_csv(getPath(finalpath,tableHas+".csv"),header = None, float_precision = "high")
    df2 = uniFormatDF(df2)
    df1 = np.array(df1)
    pk = np.array([])
    arrayCommunity = np.array([])
    communityPK = get_communityPK(elem)
    for index, row in df2.iterrows():
        for row2 in df1:
            if(row[x] == row2[1]):
                pk = np.append(pk,row2[0])
    pk = pd.DataFrame(pk)
    for index, row in pk.iterrows():
        arrayCommunity = np.append(arrayCommunity,communityPK[0][0])
    arrayCommunity = pd.DataFrame(arrayCommunity)
    result = concatDF(pk,concatDF(arrayCommunity,df2))
    result = result.drop(result.columns[[y]], axis = 1)
    return result

    
def get_specialTable(table,tableHas):
    result = pd.DataFrame()
    if(tableHas == 'se_expensetype_has_community'):
        result = get_specialTableFKs(table,tableHas,2,4)
    else:
        if(tableHas == 'se_worktype_has_community' or tableHas == 'u_area_has_community'):
            result = get_specialTableFKs(table,tableHas,3,5)
        else:
            result = get_specialTableFKs(table,tableHas,1,3)
    return result


def specialTable(cad):
    result = False
    if(cad == "camp_naturalhazard_has_camp"):
        result = True
    else:
        if(cad == "inf_appliance_has_community"):
            result = True
        else:
            if(cad == "inf_generationsource_has_community"):
                result = True
            else:
                if(cad == "se_expensetype_has_community"):
                    result = True
                else:
                    if(cad == "fs_foodsource_has_community"):
                        result = True
                    else:
                        if(cad == "fs_importantmeal_has_community"):
                            result = True
                        else:
                            if(cad == "fs_timesperday_has_community"):
                                result = True
                            else:
                                if(cad == "fs_typicalplate_has_community"):
                                    result = True
                                else:
                                    if(cad == "inf_mobilityway_has_community"):
                                        result = True
                                    else:
                                        if(cad == "s_app_has_community"):
                                            result = True
                                        else:
                                            if(cad == "se_priority_has_community"):
                                                result = True
                                            else:
                                                if(cad == "se_safetyplace_has_community"):
                                                    result = True
                                                else:
                                                    if(cad == "se_worktype_has_community"):
                                                        result = True
                                                    else:
                                                        if(cad == "u_area_has_community"):
                                                            result = True
                                                        else:
                                                            if(cad == "camp_energysource_has_camp"):
                                                                result = True
    return result

def get_tablePK(table):
    cursor.execute("SELECT * FROM "+table)
    df1 = uniFormatTable(pd.DataFrame(cursor.fetchall()))
    df2 = pd.read_csv(getPath(finalpath,table+".csv"), float_precision = "high")
    df2 = uniFormatDF(df2)
    df1 = np.array(df1)
    pk = np.array([])
    for index, row in df2.iterrows():
        if(serviceTable(table) == False):
            for row2 in df1:
                if((np.equal(np.array(row),np.array(row2[1:]))).all()):
                    pk = np.append(pk,row2[0])
        else:
            if(isEducationalCenter(table)):
                for row2 in df1:
                    if((np.equal(np.array(row[:-2]),row2[1:-3])).all()):
                        pk = np.append(pk,row2[0])
            else:
                if(isOtherCenter(table) == False):
                    for row2 in df1:
                        if((np.equal(np.array(row[:-1]),row2[1:-2])).all()):
                            pk = np.append(pk,row2[0])
                else:
                    for row2 in df1:
                        if((np.equal(np.array(row[:-1]),row2[1:-1])).all()):
                            pk = np.append(pk,row2[0])
    return pd.DataFrame(pk)

tablesNM = pd.read_csv("NMtablesCamp.csv",header = None)
tablesNM = np.array(tablesNM)
tables = np.array([])
originTables = np.array([])
for column in tablesNM:
    for elem in column:
        x = getTableName(elem)
        print(x)
        if(is_non_zero_file(getPath(finalpath,x+".csv"))):
            if(specialTable(elem) == False):
                tablePK = get_tablePK(x)
                communityPK = get_communityPK(elem)
                arrayCommunity = np.array([])
                for index, row in tablePK.iterrows():
                    arrayCommunity = np.append(arrayCommunity,communityPK[0][0])
                nmTableFK = concatDF(tablePK,pd.DataFrame(arrayCommunity))
                if(os.path.isfile(finalpath+"/"+elem+".csv")):
                    if(is_non_zero_file(getPath(finalpath,elem+".csv"))):
                        df = np.array(pd.read_csv(finalpath+"/"+elem+".csv"))
                        df = pd.DataFrame(df)
                        nmTableFK = concatDF(nmTableFK,df)
            else:
                nmTableFK = get_specialTable(x,elem)                
            print(nmTableFK)
            mkCSV(nmTableFK,elem+".csv")
            print(elem+".csv")
                     

camp_energysource
     0    1      2
0    1  5.0    nan
1   16  5.0    nan
2    2  5.0    nan
3   17  5.0    nan
4    3  5.0  100.0
5   18  5.0    yes
6    4  5.0   10.0
7   19  5.0    yes
8    5  5.0   50.0
9   20  5.0    nan
10   6  5.0    nan
11  21  5.0   25.0
12   7  5.0    nan
13  22  5.0    nan
14   8  5.0    nan
15  23  5.0    nan
16   1  5.0    nan
17  16  5.0    nan
18   2  5.0    5.0
19  17  5.0    nan
20   3  5.0    nan
21  18  5.0    nan
22   4  5.0  750.0
23  19  5.0    yes
24   5  5.0  200.0
25  20  5.0     no
26   6  5.0    nan
27  21  5.0    nan
28   7  5.0    nan
29  22  5.0    nan
..  ..  ...    ...
34   2  5.0    nan
35  17  5.0    nan
36   3  5.0    NaN
37  18  5.0    NaN
38   4  5.0    NaN
39  19  5.0    NaN
40   5  5.0    NaN
41  20  5.0    NaN
42   6  5.0    NaN
43  21  5.0    NaN
44   7  5.0    NaN
45  22  5.0    NaN
46   8  5.0    NaN
47  23  5.0    NaN
48   1  5.0    NaN
49  16  5.0    NaN
50   2  5.0    NaN
51  17  5.0    NaN
52   3  5.0    NaN
53  18  5.0  

       0    1
0      2  2.0
1      4  2.0
2      5  2.0
3      7  2.0
4      9  2.0
5     10  2.0
6     12  2.0
7     14  2.0
8     15  2.0
9     16  2.0
10    20  2.0
11    21  2.0
12    22  2.0
13    23  2.0
14    24  2.0
15    26  2.0
16    27  2.0
17    29  2.0
18    32  2.0
19    65  2.0
20    67  2.0
21    68  2.0
22    70  2.0
23    72  2.0
24    73  2.0
25    75  2.0
26    77  2.0
27    78  2.0
28    79  2.0
29    83  2.0
..   ...  ...
838   37  2.0
839   71  2.0
840   93  2.0
841   97  2.0
842   99  2.0
843  100  2.0
844   33  2.0
845   35  2.0
846   96  2.0
847   98  2.0
848    8  2.0
849   30  2.0
850   34  2.0
851   36  2.0
852   37  2.0
853   71  2.0
854   93  2.0
855   97  2.0
856   99  2.0
857  100  2.0
858    8  2.0
859   30  2.0
860   34  2.0
861   36  2.0
862   37  2.0
863   71  2.0
864   93  2.0
865   97  2.0
866   99  2.0
867  100  2.0

[868 rows x 2 columns]
inf_sanitationsystemquality_has_community.csv
s_app
    0    1         2
0   1  2.0      Used
1  16  2.0  Ne